
On previous pages we've gone through how to persist databases from one container to another by using: -
> [Named Volumes](https://github.com/dbafromthecold/SqlServerAndContainersGuide/wiki/Persisting-data-using-named-volumes)<br>
> [Data Volume Containers](https://github.com/dbafromthecold/SqlServerAndContainersGuide/wiki/Persisting-data-using-data-volume-containers)
Another way to is mount a volume from the Docker host into a container.
Let's create a couple of directories on the host: -



In [0]:
    mkdir C:\sqlserver\data
    mkdir C:\sqlserver\log



![](../images/3.PersistingData/BindMounts/1.CreateVolumesOnHost.png)
Now, run a container: -



In [0]:
    docker container run -d  `
    --publish 15789:1433 `
    --volume C:\sqlserver\data:/opt/sqlserver/data `
    --volume C:\sqlserver\log:/opt/sqlserver/log `
    --env ACCEPT_EULA=Y `
    --env MSSQL_SA_PASSWORD=Testing1122 `
    --env MSSQL_DATA_DIR=/opt/sqlserver/data `
    --env MSSQL_LOG_DIR=/opt/sqlserver/log `
    --name sqlcontainer1 `
    mcr.microsoft.com/mssql/server:2019-CU5-ubuntu-18.04



This will map the volumes on the host to /opt/sqlserver/data and /opt/sqlserver/log in the container. 
We're also using environment variables to set the SQL Server default data and log locations to those volumes in the container.
Let's confirm that the container is up: -



In [0]:
    docker container ls -a



![](../images/3.PersistingData/BindMounts/2.RunSqlContainer.png)
And now let's create a database: -



In [0]:
    mssql-cli -S localhost,15789 -U sa -P Testing1122 -Q "CREATE DATABASE [testdatabase]"



No need to set the data and log file location as we've set the defaults to the volumes in the container that are mapped to the volumes on the host.
We can check the database files by running: -



In [0]:
    mssql-cli -S localhost,15789 -U sa -P Testing1122 -Q "USE [testdatabase]; EXEC sp_helpfile;"



![](../images/3.PersistingData/BindMounts/3.ConfirmDatabaseFileLocation.png)
And we can confirm the files on the host by running: -



In [0]:
    ls C:\sqlserver\data
    ls C:\sqlserver\log



![](../images/3.PersistingData/BindMounts/4.ConfirmDatabaseFilesOnHost.png)
And there are the database files on the host!
Ok, blow the container away: -



In [0]:
    docker rm sqlcontainer1 -f



And spin up a new container, remapping the volumes from the host: -



In [0]:
    docker container run -d  `
    --publish 15799:1433 `
    --volume C:\sqlserver\data:/opt/sqlserver/data `
    --volume C:\sqlserver\log:/opt/sqlserver/log `
    --env ACCEPT_EULA=Y `
    --env MSSQL_SA_PASSWORD=Testing1122 `
    --env MSSQL_DATA_DIR=/opt/sqlserver/data `
    --env MSSQL_LOG_DIR=/opt/sqlserver/log `
    --name sqlcontainer2 `
    mcr.microsoft.com/mssql/server:2019-CU5-ubuntu-18.04



Confirm the container is up and running: -



In [0]:
     docker container ls -a



![](../images/3.PersistingData/BindMounts/5.RunSecondSqlContainer.png)
Cool. Ok, now as we didn't persist the master database location we'll need to manually attach the database into SQL in the new container: -



In [0]:
     mssql-cli -S localhost,15799 -U sa -P Testing1122 `
     -Q "CREATE DATABASE [testdatabase1] ON PRIMARY (NAME='testdatabase1',FILENAME='/var/opt/sqlserver/data testdatabase1.mdf') LOG ON (NAME='testdatabase1_log',FILENAME='/var/opt/sqlserver/log/testdatabase1_log.ldf') FOR ATTACH;"



And now check that the database is there: -



In [0]:
     mssql-cli -S localhost,15799 -U sa -P Testing1122 -Q "SELECT [name] FROM sys.databases;"
